In [1]:
%%capture
%run full_setup.py

In [3]:
from tfidf_corpus_dictionary import get_tfidf_tokendocs_corpus_dict
from gensim.models import LdaModel, LsiModel, CoherenceModel
from sklearn.decomposition import NMF, PCA
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
import numpy as np
from scipy import sparse

In [4]:
tfidf_matrix, feature_names, tokenized_docs, corpus, dictionary = get_tfidf_tokendocs_corpus_dict(df, max_df=0.5, min_df=5, max_features=5000)

In [7]:
from coherence_topics import coherence_topics

In [9]:
evaluation = dict()
models = ['LDA', 'LSA', 'NMF', 'PCA', 'RP']

for mod in models:
    metrics = coherence_topics(model_name=mod, corpus=corpus, dictionary=dictionary,
                               texts=tokenized_docs, feature_names=feature_names, tfidf=tfidf_matrix)
    evaluation[mod] = metrics

In [10]:
evaluation['PCA']

[(5, 0.6126217078978584),
 (10, 0.5324411068937985),
 (15, 0.4665291883941073),
 (20, 0.4358455639844679),
 (50, 0.34205080349787276)]

In [19]:
n_components = 5
# Fit an PCA model
pca = PCA(n_components=n_components)

# Convert sparse matrix to dense, if necessary
tfidf_matrix_dense = tfidf_matrix.todense() if sparse.issparse(tfidf_matrix) else tfidf_matrix

# Convert to numpy array
tfidf_matrix_array = np.asarray(tfidf_matrix_dense)

# Centering
mean_tfidf = np.mean(tfidf_matrix_array, axis=0)  # Calculate the mean of each column
centered_tfidf_matrix = tfidf_matrix_array - mean_tfidf

 # Fit PCA on the centered dense matrix
pca_matrix = pca.fit_transform(centered_tfidf_matrix)

pca_matrix 

array([[ 0.2145996 , -0.18889009, -0.14162026, -0.00383668, -0.09508634],
       [ 0.34976799, -0.2617644 , -0.18140826, -0.00386019, -0.17265026],
       [ 0.08335612, -0.04238742, -0.03393469, -0.00220794, -0.03110297],
       ...,
       [ 0.12438077, -0.07619421, -0.09234804, -0.0050619 , -0.06726379],
       [ 0.1074545 , -0.04445373, -0.03229813,  0.00713503, -0.02749549],
       [ 0.1212295 , -0.06564494, -0.02712155, -0.0048444 , -0.00654507]])

In [23]:
from gensim import corpora, matutils
# Convert each row of the PCA matrix to a sparse format (list of tuples)
pca_corp = [list(zip(range(len(row)), row)) for row in pca_matrix]

# Now create a Gensim corpus from this list of sparse vectors
pca_corpus = [matutils.sparse2full(doc, length=pca_matrix.shape[1]) for doc in pca_corp]


In [25]:
# Apply LDA on the reduced dimentinality pca_matrix

model = LdaModel(corpus=pca_corpus, id2word=dictionary, num_topics = 5,
                                alpha='symmetric', eta='auto', passes=5, random_state=1)

IndexError: invalid index to scalar variable.